# Параметрическое исследование экспоненциального роста

## Активация проекта и загрузка пакетов

In [1]:
using DrWatson
@quickactivate "project"
using DifferentialEquations
using DataFrames
using Plots
using JLD2
using BenchmarkTools

Установка каталогов

In [2]:
script_name = splitext(basename(PROGRAM_FILE))[1]
mkpath(plotsdir(script_name))
mkpath(datadir(script_name))

"/home/vvmalyuga/work/study/2026-1/2026-1==study--simulation-modeling/labs/lab01/project/data"

## Определение модели
Модель: du/dt = α * u

In [3]:
function exponential_growth!(du, u, p, t)
    α = p.α
    du[1] = α * u[1]
end

exponential_growth! (generic function with 1 method)

## Определение параметров в Dict
Базовый набор параметров (один эксперимент)

In [4]:
base_params = Dict(
    :u0 => [1.0],
    :α => 0.3,
    :tspan => (0.0, 10.0),
    :solver => Tsit5(),
    :saveat => 0.1,
    :experiment_name => "base_experiment"
)

println("Базовые параметры эксперимента:")
for (key, value) in base_params
    println(" $key = $value")
end

Базовые параметры эксперимента:
 α = 0.3
 u0 = [1.0]
 saveat = 0.1
 solver = Tsit5{typeof(OrdinaryDiffEqCore.trivial_limiter!), typeof(OrdinaryDiffEqCore.trivial_limiter!), Static.False}(OrdinaryDiffEqCore.trivial_limiter!, OrdinaryDiffEqCore.trivial_limiter!, static(false))
 experiment_name = base_experiment
 tspan = (0.0, 10.0)


## Функция-обертка для запуска одного эксперимента

In [5]:
function run_single_experiment(params::Dict)
    @unpack u0, α, tspan, solver, saveat = params
    prob = ODEProblem(exponential_growth!, u0, tspan, (α=α,))
    sol = solve(prob, solver; saveat=saveat)
    final_population = last(sol.u)[1]
    doubling_time = log(2) / α
    return Dict(
        "solution" => sol,
        "time_points" => sol.t,
        "population_values" => first.(sol.u),
        "final_population" => final_population,
        "doubling_time" => doubling_time,
        "parameters" => params
    )
end

run_single_experiment (generic function with 1 method)

## Запуск базового эксперимента

In [6]:
data, path = produce_or_load(
    datadir(script_name, "single"),
    base_params,
    run_single_experiment,
    prefix = "exp_growth",
    tag = false,
    verbose = true
)

println("\nРезультаты базового эксперимента:")
println(" Финальная популяция: ", data["final_population"])
println(" Время удвоения: ", round(data["doubling_time"]; digits=2))
println(" Файл результатов: ", path)

[ Info: File /home/vvmalyuga/work/study/2026-1/2026-1==study--simulation-modeling/labs/lab01/project/data/single/exp_growth_experiment_name=base_experiment_saveat=0.1_α=0.3.jld2 does not exist. Producing it now...
[ Info: File /home/vvmalyuga/work/study/2026-1/2026-1==study--simulation-modeling/labs/lab01/project/data/single/exp_growth_experiment_name=base_experiment_saveat=0.1_α=0.3.jld2 saved.



Результаты базового эксперимента:
 Финальная популяция: 20.085448516186737
 Время удвоения: 2.31
 Файл результатов: /home/vvmalyuga/work/study/2026-1/2026-1==study--simulation-modeling/labs/lab01/project/data/single/exp_growth_experiment_name=base_experiment_saveat=0.1_α=0.3.jld2


## Визуализация базового эксперимента

In [7]:
p1 = plot(data["time_points"], data["population_values"],
          label="α = $(base_params[:α])",
          xlabel="Время, t",
          ylabel="Популяция, u(t)",
          title="Экспоненциальный рост (базовый эксперимент)",
          lw=2,
          legend=:topleft,
          grid=true)

savefig(plotsdir(script_name, "single_experiment.png"))

"/home/vvmalyuga/work/study/2026-1/2026-1==study--simulation-modeling/labs/lab01/project/plots/single_experiment.png"

## Параметрическое сканирование
Сетка параметров для сканирования

In [8]:
param_grid = Dict(
    :u0 => [[1.0]],
    :α => [0.1, 0.3, 0.5, 0.8, 1.0],
    :tspan => [(0.0, 10.0)],
    :solver => [Tsit5()],
    :saveat => [0.1],
    :experiment_name => ["parametric_scan"]
)

all_params = dict_list(param_grid)

println("\n" * "="^60)
println("ПАРАМЕТРИЧЕСКОЕ СКАНИРОВАНИЕ")
println("Всего комбинаций параметров: ", length(all_params))
println("Исследуемые значения α: ", param_grid[:α])
println("="^60)


ПАРАМЕТРИЧЕСКОЕ СКАНИРОВАНИЕ
Всего комбинаций параметров: 5
Исследуемые значения α: [0.1, 0.3, 0.5, 0.8, 1.0]


## Запуск всех экспериментов

In [9]:
all_results = []
all_dfs = []

for (i, params) in enumerate(all_params)
    println("Прогресс: $i/$(length(all_params)) | α = $(params[:α])")
    data, path = produce_or_load(
        datadir(script_name, "parametric_scan"),
        params,
        run_single_experiment,
        prefix = "scan",
        tag = false,
        verbose = false
    )

    result_summary = merge(
        params,
        Dict(
            :final_population => data["final_population"],
            :doubling_time => data["doubling_time"],
            :filepath => path
        )
    )
    push!(all_results, result_summary)

    df = DataFrame(
        t = data["time_points"],
        u = data["population_values"],
        α = fill(params[:α], length(data["time_points"]))
    )
    push!(all_dfs, df)
end

Прогресс: 1/5 | α = 0.1
Прогресс: 2/5 | α = 0.3
Прогресс: 3/5 | α = 0.5
Прогресс: 4/5 | α = 0.8
Прогресс: 5/5 | α = 1.0


## Анализ и визуализация результатов

In [10]:
results_df = DataFrame(all_results)
println("\nСводная таблица результатов:")
println(results_df[!, [:α, :final_population, :doubling_time]])


Сводная таблица результатов:
5×3 DataFrame
 Row │ α        final_population  doubling_time 
     │ Float64  Float64           Float64       
─────┼──────────────────────────────────────────
   1 │     0.1           2.71828       6.93147
   2 │     0.3          20.0854        2.31049
   3 │     0.5         148.409         1.38629
   4 │     0.8        2980.57          0.866434
   5 │     1.0       22021.0           0.693147


Сравнительный график всех траекторий

In [11]:
p2 = plot(size=(800, 500), dpi=150)
for params in all_params
    data, _ = produce_or_load(
        datadir(script_name, "parametric_scan"),
        params,
        run_single_experiment,
        prefix = "scan"
    )
    plot!(p2, data["time_points"], data["population_values"],
          label="α = $(params[:α])",
          lw=2,
          alpha=0.8)
end

plot!(p2,
      xlabel="Время, t",
      ylabel="Популяция, u(t)",
      title="Параметрическое исследование: влияние α на рост",
      legend=:topleft,
      grid=true)

savefig(plotsdir(script_name, "parametric_scan_comparison.png"))

"/home/vvmalyuga/work/study/2026-1/2026-1==study--simulation-modeling/labs/lab01/project/plots/parametric_scan_comparison.png"

График зависимости времени удвоения от α

In [12]:
p3 = plot(results_df.α, results_df.doubling_time,
          seriestype=:scatter,
          label="Численное решение",
          xlabel="Скорость роста, α",
          ylabel="Время удвоения, t₂",
          title="Зависимость времени удвоения от α",
          markersize=8,
          markercolor=:red,
          legend=:topright)

α_range = 0.1:0.01:1.0
plot!(p3, α_range, log(2) ./ α_range,
      label="Теория: t₂ = ln(2)/α",
      lw=2,
      linestyle=:dash,
      linecolor=:blue)

savefig(plotsdir(script_name, "doubling_time_vs_alpha.png"))

"/home/vvmalyuga/work/study/2026-1/2026-1==study--simulation-modeling/labs/lab01/project/plots/doubling_time_vs_alpha.png"

## Бенчмаркинг

In [13]:
println("\n" * "="^60)
println("Бенчмаркинг для разных значений α")
println("="^60)

benchmark_results = []
for α_value in param_grid[:α]
    bench_params = Dict(
        :u0 => [1.0],
        :α => α_value,
        :tspan => (0.0, 10.0),
        :solver => Tsit5(),
        :saveat => 0.1
    )

    function benchmark_run()
        prob = ODEProblem(exponential_growth!,
                          bench_params[:u0],
                          bench_params[:tspan],
                          (α=bench_params[:α],))
        return solve(prob, bench_params[:solver]; saveat=bench_params[:saveat])
    end

    println("\nБенчмарк для α = $α_value:")
    b = @benchmark $benchmark_run() samples=100 evals=1
    push!(benchmark_results, (α=α_value, time=median(b).time/1e9))
    println(" Среднее время: ", round(median(b).time/1e9; digits=4), " сек")
end

bench_df = DataFrame(benchmark_results)

p4 = plot(bench_df.α, bench_df.time,
          seriestype=:scatter,
          label="Время вычисления",
          xlabel="Скорость роста, α",
          ylabel="Время вычисления, сек",
          title="Зависимость времени вычисления от α",
          markersize=8,
          markercolor=:green,
          legend=:topleft)

savefig(plotsdir(script_name, "computation_time_vs_alpha.png"))


Бенчмаркинг для разных значений α

Бенчмарк для α = 0.1:
 Среднее время: 0.0002 сек

Бенчмарк для α = 0.3:
 Среднее время: 0.0002 сек

Бенчмарк для α = 0.5:
 Среднее время: 0.0002 сек

Бенчмарк для α = 0.8:
 Среднее время: 0.0002 сек

Бенчмарк для α = 1.0:
 Среднее время: 0.0002 сек


"/home/vvmalyuga/work/study/2026-1/2026-1==study--simulation-modeling/labs/lab01/project/plots/computation_time_vs_alpha.png"

## Сохранение всех результатов

In [14]:
@save datadir(script_name, "all_results.jld2") base_params param_grid all_params results_df bench_df
@save datadir(script_name, "all_plots.jld2") p1 p2 p3 p4

println("\n" * "="^60)
println("ЛАБОРАТОРНАЯ РАБОТА ЗАВЕРШЕНА")
println("="^60)
println("\nРезультаты сохранены в:")
println(" • data/$(script_name)/single/ - базовый эксперимент")
println(" • data/$(script_name)/parametric_scan/ - параметрическое сканирование")
println(" • data/$(script_name)/all_results.jld2 - сводные данные")
println(" • plots/$(script_name)/ - все графики")
println("\nДля анализа результатов используйте:")
println(" using JLD2, DataFrames")
println(" @load \"data/$(script_name)/all_results.jld2\"")
println(" println(results_df)")


ЛАБОРАТОРНАЯ РАБОТА ЗАВЕРШЕНА

Результаты сохранены в:
 • data//single/ - базовый эксперимент
 • data//parametric_scan/ - параметрическое сканирование
 • data//all_results.jld2 - сводные данные
 • plots// - все графики

Для анализа результатов используйте:
 using JLD2, DataFrames
 @load "data//all_results.jld2"
 println(results_df)
